In [1]:
import fastparquet
import numpy as np

In [2]:
pf = fastparquet.ParquetFile("metadetect_desdmv4_cutsv3_jk")

In [5]:
pf.columns

['uid',
 'tilename',
 'slice_id',
 'ra',
 'dec',
 'x',
 'y',
 'mfrac',
 'mfrac_img',
 'nepoch_g',
 'nepoch_r',
 'nepoch_i',
 'nepoch_z',
 'psfrec_g_1',
 'psfrec_g_2',
 'psfrec_T',
 'gauss_s2n',
 'gauss_g_1',
 'gauss_g_2',
 'gauss_g_cov_1_1',
 'gauss_g_cov_1_2',
 'gauss_g_cov_2_2',
 'gauss_T_err',
 'gauss_T_ratio',
 'gauss_psf_T',
 'pgauss_T_err',
 'pgauss_T',
 'pgauss_psf_T',
 'pgauss_band_flux_g',
 'pgauss_band_flux_r',
 'pgauss_band_flux_i',
 'pgauss_band_flux_z',
 'pgauss_band_flux_err_g',
 'pgauss_band_flux_err_r',
 'pgauss_band_flux_err_i',
 'pgauss_band_flux_err_z']

In [17]:
%%time

dd = pf.to_pandas(
    columns=["ra", "dec", "gauss_g_1", "gauss_g_2"],
    filters=[[("mdet_step", "==", "noshear")]],
    index=False,
).to_records()

dd = np.array(dd, dtype=dd.dtype.descr)

CPU times: user 8.19 s, sys: 13.4 s, total: 21.6 s
Wall time: 1min 4s


In [18]:
%%time

dd = pf.to_pandas(
    columns=["ra", "dec", "gauss_g_1", "gauss_g_2"],
    filters=[[("mdet_step", "==", "noshear"), ("patch_num", "==", 20)]],
    index=False,
).to_records()

dd = np.array(dd, dtype=dd.dtype.descr)

CPU times: user 280 ms, sys: 575 ms, total: 855 ms
Wall time: 6.04 s
